In [2]:
#Comparing HOG, Random Projection, Min Hash running time with CIFAR10
from sklearn.datasets import fetch_openml
import numpy as np 
from sklearn.decomposition import PCA
from sklearn.random_projection import GaussianRandomProjection
import time
from datasketch import MinHash, MinHashLSH


In [15]:
def random_projection_lsh(data, n_hash, n_dimensions):
    random_vectors = np.random.randn(n_hash, n_dimensions)
    projections = np.dot(data, random_vectors.T)
    hash_codes = (projections > 0).astype(int)
    return hash_codes 

def time_random_projection_query(query, hash_codes):
    start_time = time.time()
    query_hash = random_projection_lsh(query.reshape(1,-1))
    hamming_distances = np.sum(query_hash != hash_codes, axis = 1)
    sorted_indices = np.argsort(hamming_distances)
    end_time = time.time()
    return sorted_indices, end_time- start_time



In [11]:
def minhash_lsh(data, num_perm=128):
    # Create an LSH index using min hash for jaccard similarity
    lsh = MinHashLSH(threshold=0.5, num_perm=num_perm)
    minhashes = []
    for i, vector in enumerate(data):
        # Convert dense vector to a set of indices with non zero values
        vector_set = set(np.nonzero(vector)[0])
        m = MinHash(num_perm=num_perm)
        for idx in vector_set:
            m.update(str(idx).encode('utf8'))
        minhashes.append(m)
        lsh.insert(f"data_{i}", m)
    return lsh, minhashes
def time_minhash_query(query, lsh, num_perm=128):
    query_set = set(np.nonzero(query)[0])
    query_minhash = MinHash(num_perm=num_perm)
    for idx in query_set:
        query_minhash.update(str(idx).encode('utf8'))
    
    start_time = time.time()
    result = lsh.query(query_minhash)
    end_time = time.time()
    return result, end_time - start_time

In [16]:
query_index = 0
query_image = X_pca[query_index]

_, rp_query_time =time_random_projection_query(query_image)

TypeError: time_random_projection_query() missing 1 required positional argument: 'hash_codes'

In [16]:
import time
import numpy as np
from datasketch import MinHash, MinHashLSH

#load cifar10
cifar10 = fetch_openml('CIFAR_10_small')
#transform data to float, label to integers
X = cifar10.data.astype(np.float32)
y = cifar10.target.astype(np.int64)
#PCA to reduce dimensionality for faster processing 
pca = PCA(n_components = 100)
X_pca = pca.fit_transform(X)


n_hash = 128
n_dimensions = X_pca.shape[1]



c:\Users\xiaoy\.pyenv\pyenv-win\versions\3.11.0b4\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
#Define Random Projection LSH
def random_projection_lsh(data:list[list[float]], n_hash:int=128, seed = 1) -> list[int]:
    """"    
    
    data : a single image as a matrix of floats

    n_hash : size of the hash
    
    returns: list of binary values
    """
    np.random.seed(seed)
    n_dimensions = data.shape[1]
    random_vectors = np.random.randn(n_hash, n_dimensions)
    projections = np.dot(data, random_vectors.T)
    hash_codes = (projections > 0).astype(int)
    return hash_codes 

# Timing Random Projection Query
def time_random_projection_query(query, hash_codes, n_hash, n_dimensions):
    start_time = time.time()
    query_hash = random_projection_lsh(query.reshape(1, -1), n_hash, n_dimensions)
    hamming_distances = np.sum(query_hash != hash_codes, axis=1) #computing eucleadian distance
    sorted_indices = np.argsort(hamming_distances)
    end_time = time.time()
    return sorted_indices, end_time - start_time

# Define MinHash LSH
def minhash_lsh(data, num_perm=128):
    # Create an LSH index using min hash for Jaccard similarity, intersection/union of two item sets
    # Greater than 0.5 are considered similar, 128 is number of permutations to approximate jaccard similarity
    lsh = MinHashLSH(threshold=0.5, num_perm=num_perm) 
    minhashes = []
    for i, vector in enumerate(data): 
        #Convert each vector to a set of indices with non-zero values
        vector_set = set(np.nonzero(vector)[0])  
        m = MinHash(num_perm=num_perm) #sets are transoformed into MinHash signitures *based on jaccard similarity
        for idx in vector_set:
            m.update(str(idx).encode('utf8')) # Add elements of minhash object m, index non-zero transformed into hashable strings
        minhashes.append(m)
        lsh.insert(f"data_{i}", m) # Add minhash object to lsh index
    return lsh, minhashes

# Timing MinHash Query
def time_minhash_query(query, lsh, num_perm=128):
    query_set = set(np.nonzero(query)[0])
    query_minhash = MinHash(num_perm=num_perm)
    for idx in query_set:
        query_minhash.update(str(idx).encode('utf8')) 
    start_time = time.time()
    result = lsh.query(query_minhash)
    end_time = time.time()
    return result, end_time - start_time

# Sample Data and Parameters
n_hash = 128
n_dimensions = X_pca.shape[1]

# Prepare the data for Random Projection
hash_codes = random_projection_lsh(X_pca, n_hash, n_dimensions)

# Prepare the data for MinHash
lsh, minhashes = minhash_lsh(X_pca, num_perm=128)

# Query Comparison
query_index = 5
query_image = X_pca[query_index]


In [6]:
# Random Projection LSH Query
_, rp_query_time = time_random_projection_query(query_image, hash_codes, n_hash, n_dimensions)

# MinHash Query
_, minhash_query_time = time_minhash_query(query_image, lsh)

# Print Results
print(f"Random Projection Query Time: {rp_query_time} seconds")
print(f"MinHash Query Time: {minhash_query_time} seconds")

Random Projection Query Time: 0.015003204345703125 seconds
MinHash Query Time: 0.06342077255249023 seconds


In [15]:
X_pca.shape

(20000, 100)

In [3]:
X_pca

NameError: name 'X_pca' is not defined